In [ ]:
import os, folium, pandas as pd, numpy as np
from datetime import date, datetime, timedelta
from collections import Counter
from sqlalchemy import create_engine
from sklearn.cluster import KMeans, HDBSCAN
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt, seaborn as sns

## parameters

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

In [ ]:
cdef = 'tl7_10m'# 'tl5_10m' 'tl6_10m' 'tl7_10m' 'tl8_10m' 'tl8_60m'
cdef_alt = '16m_10min'# tl5: 62 ... tl7: 16   tl8: 8

## database connection

In [ ]:
# database credentials
db_usr, db_pwd = os.getenv('DB_USR'), os.getenv('DB_PWD') # your database user name and password
# database login
host, port, db = 'nc-health-data-prod.cluster-ccsgl7rk4urn.eu-central-1.rds.amazonaws.com', 5432, 'master'

In [ ]:
# for queries with output
engine = create_engine('postgresql://'+db_usr+':'+db_pwd+'@'+host+':'+str(port)+'/'+db)
conn = engine.connect()

In [ ]:
conn.close()

## load data

In [ ]:
# load mass event data
match_data = pd.read_csv('output/00_event_data.csv')
match_data['day'] = [d.date() for d in pd.to_datetime(match_data.day)]

In [ ]:
vac_data = [
    ['Berlin', date(2024,7,18), date(2024,8,30)],
    ['Dortmund', date(2024,7,8), date(2024,8,20)],
    ['Düsseldorf', date(2024,7,8), date(2024,8,20)],
    ['Frankfurt am Main', date(2024,7,15), date(2024,8,23)],
    ['Gelsenkirchen', date(2024,7,8), date(2024,8,20)],
    ['Hamburg', date(2024,7,18), date(2024,8,28)],
    ['Köln', date(2024,7,8), date(2024,8,20)],
    ['Leipzig', date(2024,6,20), date(2024,8,2)],
    ['München', date(2024,7,29), date(2024,9,9)],
    ['Stuttgart', date(2024,7,25), date(2024,9,7)],
]
vac_data = pd.DataFrame(vac_data, columns=['city','day_start','day_end'])
vac_data

## did pair timeseries: classification of did pairs as stable vs. random contacts

#cdef_cn = '_'+cdef if cdef == 'tl8_2min' else ''
query = f"""
    with cn_tmp as (
    	select
                  tl{cdef[2]}
                , geopoint
                , "day"
                , stime
                , dids
                , sources
     			, bool_or(u.dist_stad < csa.radius_in_meter) as in_stadium
                , min(u.area_id) as area_id
            from covid_network_sdkv6_{cdef}, unnest(area_ids, dist_stads) u(area_id, dist_stad)
            left join cluster_search_areas_v2 csa on csa.area_id = u.area_id
            where
                    "day" between '{di}' and '{df}'
            group by 1,2,3,4,5,6
    ),
    cn as (
        select
                  tl{cdef[2]}
                , "day"
                , stime
                , dids
                , sources
    			, in_stadium
                , area_id
                , st_x(st_transform(geopoint, 4326)) as lon, st_y(st_transform(geopoint, 4326)) as lat
        from cn_tmp
    )
    select *
    from cn
"""
pd.DataFrame(pd.read_sql_query(query, conn))

In [ ]:
query = f"""
    select "day", stime, dids, st_x(st_transform(geopoint, 4326)) as lon, st_y(st_transform(geopoint, 4326)) as lat, tl7--, array_length(inside_building, 1) > 0 as inside_building
    from covid_network_sdkv6_tl7_10m as cn
    --join txc_dt_grid_1000m as tx on cn.tile_id = tx.tile_id
    where "day" between '{di}' and '{df}'
"""
data = pd.DataFrame(pd.read_sql_query(query, conn))
data

In [ ]:
data_sv = data.copy(deep=True)

In [ ]:
data = data.explode('dids').reset_index(drop=True)
data = data.drop_duplicates()
data = data.merge(data.drop(columns=['lon','lat']), on=['day','stime','tl7'])#,'tl7','inside_building','lon','lat'])
data = data[data.dids_x != data.dids_y]
pairs = []
for did1, did2 in zip(data.dids_x, data.dids_y):
    pair = f'{did1}_{did2}' if did1 < did2 else f'{did2}_{did1}'
    #print(did1, did2, pair)
    pairs.append(pair)
data.loc[:,'pair'] = pairs
data = data.drop(columns=['dids_x','dids_y','tl7'])
data = data.drop_duplicates()
dmin = data.day.min()
#data['tt'] = data.day.apply(lambda d: (d-dmin).days)*24 + data.stime.dt.hour
data['tt'] = data.day.apply(lambda d: (d-dmin).days)*24*6 + data.stime.dt.hour*6 + (data.stime.dt.minute//10)
print(data.tt.max(), ((data.day.max()-dmin).days+1)*24, ((data.day.max()-dmin).days+1)*720)
data

In [ ]:
data.to_csv('output/08_follow_didpairs.csv', index=False)

In [ ]:
data = pd.read_csv('output/08_follow_didpairs.csv')
data['day'] = [d.date() for d in pd.to_datetime(data.day)]
data['stime'] = pd.to_datetime(data.stime)

In [ ]:
data['tt2'] = data.day.apply(lambda d: (d-dmin).days)*24*6 + data.stime.dt.hour*6 + (data.stime.dt.minute//10)
print(data.tt2.min(), data.tt2.max())

In [ ]:
(data.day.max()-dmin).days*24*6 + (24-1)*6 + (6-1)

In [ ]:
# stability: 0 = random pair, 1 = stable pair
randstab = pd.DataFrame(data.groupby('pair').day.apply(lambda x: 0 if len(set(x))==1 else 1)).reset_index()
randstab = randstab.rename(columns={'tt':'stability','day':'stability','tt2':'stability'})
randstab

In [ ]:
# number of pairs in each category tt2
randstab.groupby('stability').pair.apply(len)

In [ ]:
# number of pairs in each category tt
randstab.groupby('stability').pair.apply(len)

In [ ]:
# number of pairs in each category day
randstab.groupby('stability').pair.apply(len)

In [ ]:
# total contact duration in each category tt2
data.merge(randstab, on='pair').groupby(['stability','tt2']).pair.apply(lambda x: len(set(x))).reset_index()\
                               .groupby(['stability']).pair.apply(np.sum)

In [ ]:
# total contact duration in each category tt
data.merge(randstab, on='pair').groupby(['stability','tt2']).pair.apply(lambda x: len(set(x))).reset_index()\
                               .groupby(['stability']).pair.apply(np.sum)

In [ ]:
# total contact duration in each category day
data.merge(randstab, on='pair').groupby(['stability','tt2']).pair.apply(lambda x: len(set(x))).reset_index()\
                               .groupby(['stability']).pair.apply(np.sum)

## contact number and duration per category for Germany

In [ ]:
# number of pairs in each category by day
connr = data.merge(randstab, on='pair').groupby(['day','stability']).pair.apply(lambda x: len(set(x))).reset_index()
connr = connr.rename(columns={'pair':'nr'})
connr

In [ ]:
connr[connr.stability==0].nr.sum()

In [ ]:
# total contact duration in each category by day
condur = data.merge(randstab, on='pair').groupby(['day','stability','tt2']).pair.apply(lambda x: len(set(x))).reset_index()\
                                        .groupby(['day','stability']).pair.apply(np.sum).reset_index()
condur = condur.rename(columns={'pair':'dur'})
condur

In [ ]:
condur[condur.stability==0].dur.sum()

In [ ]:
sns.set_theme(style="ticks")

# Initialize a grid of plots with an Axes for each walk
grid = sns.FacetGrid(connr, col="stability", hue="stability", palette=sns.color_palette(),#"tab20c",
                     col_wrap=1, height=1.5, aspect=7.5)

# Draw a horizontal line to show the starting point
#grid.refline(y=0, linestyle=":")

# Draw a line plot to show the trajectory of each random walk
grid.map(plt.plot, "day", "nr", marker="o", ms=1)

# Adjust the tick positions and labels
#grid.set(xticks=[d for d in sorted(set(d2n.day)) if d.weekday()==5 and d.isocalendar()[1]%2==0],# yticks=[],
#         xlim=(d2n.day.min()-timedelta(3), d2n.day.max()+timedelta(3)), ylim=(max(.5,d2n.stime.min()), d2n.stime.max()))

# Adjust the arrangement of the plots
grid.fig.tight_layout(w_pad=1)

for ax in grid.axes.flat:
    ax.set_yscale('log')

In [ ]:
sns.set_theme(style="ticks")

# Initialize a grid of plots with an Axes for each walk
grid = sns.FacetGrid(condur, col="stability", hue="stability", palette=sns.color_palette(),#"tab20c",
                     col_wrap=1, height=1.5, aspect=7.5)

# Draw a horizontal line to show the starting point
#grid.refline(y=0, linestyle=":")

# Draw a line plot to show the trajectory of each random walk
grid.map(plt.plot, "day", "dur", marker="o", ms=1)

# Adjust the tick positions and labels
#grid.set(xticks=[d for d in sorted(set(d2n.day)) if d.weekday()==5 and d.isocalendar()[1]%2==0],# yticks=[],
#         xlim=(d2n.day.min()-timedelta(3), d2n.day.max()+timedelta(3)), ylim=(max(.5,d2n.stime.min()), d2n.stime.max()))

# Adjust the arrangement of the plots
grid.fig.tight_layout(w_pad=1)

for ax in grid.axes.flat:
    ax.set_yscale('log')

In [ ]:
conavg = connr.merge(condur, on=['day','stability'])
conavg['avg'] = conavg.dur / conavg.nr
conavg

In [ ]:
sns.set_theme(style="ticks")

# Initialize a grid of plots with an Axes for each walk
grid = sns.FacetGrid(conavg, col="stability", hue="stability", palette=sns.color_palette(),#"tab20c",
                     col_wrap=1, height=1.5, aspect=7.5)

# Draw a horizontal line to show the starting point
#grid.refline(y=0, linestyle=":")

# Draw a line plot to show the trajectory of each random walk
grid.map(plt.plot, "day", "avg", marker="o", ms=1)

# Adjust the tick positions and labels
#grid.set(xticks=[d for d in sorted(set(d2n.day)) if d.weekday()==5 and d.isocalendar()[1]%2==0],# yticks=[],
#         xlim=(d2n.day.min()-timedelta(3), d2n.day.max()+timedelta(3)), ylim=(max(.5,d2n.stime.min()), d2n.stime.max()))

# Adjust the arrangement of the plots
grid.fig.tight_layout(w_pad=1)

for ax in grid.axes.flat:
    ax.set_yscale('log')

## contact number and duration per category for host cities

In [ ]:
query = f"""
    with restricted as (
    	select cn.*
    	from tuberlin_euro24_tileid as ti
    	join covid_network_sdkv6_{cdef} as cn on cn.tile_id = ti.tile_id
        where
                "day" between '{di}' and '{df}'
    ),
    cn_tmp as (
        select
                  tl{cdef[2]}
                , geopoint
                , "day"
                , stime
                , dids
                --, sources
             	--, bool_or(u.dist_stad < csa.radius_in_meter) as in_stadium
                --, min(u.area_id) as area_id
            from restricted, unnest(area_ids, dist_stads) u(area_id, dist_stad)
            left join cluster_search_areas_v2 csa on csa.area_id = u.area_id
            --group by 1,2,3,4,7
    ),
    cities2 as (
        select osm_id, "name", max(way_area) as way_area
        from tuberlin_euro24_contour
        group by 1,2
    ),
    cities3 as (
        select c1."name", c1.way
        from tuberlin_euro24_contour as c1
        join cities2 as c2 on c1.way_area = c2.way_area
    ),
    cn as (
        select
                  tl{cdef[2]}
                , "day"
                , stime
                , unnest(dids) as did
                --, sources
        		--, in_stadium
                --, area_id
                , "name" as city
                , geopoint
        from cn_tmp
        join cities3 as c3 on st_contains(c3.way, cn_tmp.geopoint)
    ),
    cn_homes as (
        select
              tl{cdef[2]}
            , "day"
            , stime
            , array_agg(cn.did order by cn.did) as dids
            , city
            , st_x(st_transform(geopoint, 4326)) as lon, st_y(st_transform(geopoint, 4326)) as lat
            , array_agg(st_distance(st_transform(hw.weighted_centroid, 3857), geopoint) order by cn.did) as homedists
        from cn
        left join home_work_sdkv6 as hw on hw.valid_for = date_trunc('month', cn."day") + INTERVAL '1 month' - INTERVAL '1 day'
        and hw.did = cn.did and hw.place = 'home'
        group by 1,2,3,5,6,7
    )
    select *
    from cn_homes
"""

In [ ]:
data = pd.DataFrame(pd.read_sql_query(query, conn))
data

In [ ]:
data_sv = data.copy(deep=True)

In [ ]:
data = data_sv.copy(deep=True)

In [ ]:
data = data.explode(['dids','homedists']).reset_index(drop=True)
data = data.drop_duplicates()
data = data.merge(data.drop(columns=['lon','lat']), on=['day','city','stime','tl7'])#,'tl7','inside_building','lon','lat'])
data = data[data.dids_x != data.dids_y]
pairs, dhome1s, dhome2s = [], [], []
for did1, did2, dhome1, dhome2 in zip(data.dids_x, data.dids_y, data.homedists_x, data.homedists_y):
    pair = f'{did1}_{did2}' if did1 < did2 else f'{did2}_{did1}'
    #print(did1, did2, pair)
    pairs.append(pair)
    if did1 < did2:
        dhome1s.append(dhome1)
        dhome2s.append(dhome2)
    else:
        dhome1s.append(dhome2)
        dhome2s.append(dhome1)
data.loc[:,'pair'] = pairs
data.loc[:,'dhome1'] = dhome1s
data.loc[:,'dhome2'] = dhome2s
data = data.drop(columns=['dids_x','dids_y','homedists_x','homedists_y','tl7'])
data = data.drop_duplicates()
dmin = data.day.min()
#data['tt'] = data.day.apply(lambda d: (d-dmin).days)*24 + data.stime.dt.hour
data['tt'] = data.day.apply(lambda d: (d-dmin).days)*24*6 + data.stime.dt.hour*6 + (data.stime.dt.minute//10)
print(data.tt.max(), ((data.day.max()-dmin).days+1)*24, ((data.day.max()-dmin).days+1)*720)
data

In [ ]:
data.to_csv('output/08_follow_didpairs_cities.csv', index=False)

In [ ]:
data = pd.read_csv('output/08_follow_didpairs_cities.csv')
data['day'] = [d.date() for d in pd.to_datetime(data.day)]
data['stime'] = pd.to_datetime(data.stime)

In [ ]:
data

In [ ]:
data['tt2'] = data.day.apply(lambda d: (d-dmin).days)*24*6 + data.stime.dt.hour*6 + (data.stime.dt.minute//10)
print(data.tt2.min(), data.tt2.max())

In [ ]:
data

In [ ]:
# number of pairs in each category by day
connr = data.merge(randstab, on='pair').groupby(['day','city','stability']).pair.apply(lambda x: len(set(x))).reset_index()
connr = connr.rename(columns={'pair':'nr'})
connr

In [ ]:
# total contact duration in each category by day
condur = data.merge(randstab, on='pair').groupby(['day','city','stability','tt2']).pair.apply(lambda x: len(set(x))).reset_index()\
                                        .groupby(['day','city','stability']).pair.apply(np.sum).reset_index()
condur = condur.rename(columns={'pair':'dur'})
baseline = condur.groupby(['city','stability']).dur.apply(np.mean).reset_index().rename(columns={'dur':'baseline'})
condur = condur.merge(baseline, on=['city','stability'])
condur['to_baseline'] = condur.dur / condur.baseline
condur

In [ ]:
city_list = sorted(set(data.city))

In [ ]:
sns.set_theme(style="ticks")

# Initialize a grid of plots with an Axes for each walk
grid = sns.FacetGrid(connr, col="city", hue="stability", hue_order=[1,0], palette=sns.husl_palette(2),#sns.color_palette(),#"tab20c",
                     col_wrap=1, height=2, aspect=5.5, row_order=city_list, legend_out=True)

# Draw a horizontal line to show the starting point
#grid.refline(y=0, linestyle=":")

# Draw a line plot to show the trajectory of each random walk
grid.map(plt.plot, "day", "nr", marker="o", ms=1)

# Adjust the tick positions and labels
#grid.set(xticks=[d for d in sorted(set(d2n.day)) if d.weekday()==5 and d.isocalendar()[1]%2==0],# yticks=[],
#         xlim=(d2n.day.min()-timedelta(3), d2n.day.max()+timedelta(3)), ylim=(max(.5,d2n.stime.min()), d2n.stime.max()))

ylimup, ylimdown = 1.2*connr.nr.max(), 1./1.2*connr.nr.min()
axes = grid.axes.flat
holis = [date(2024,5,1), date(2024,5,9), date(2024,5,20)]
for ax, city in zip(axes, city_list):
    if True:#for ax in ax_row:
        ax.set_ylabel('detected contacts')
        #ax.plot([data_here.day.min(), data_here.day.max()], [1,1], c='gray')
        ax.set_title(city)
        ax.set_xticks(list(set(data.day)))
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in list(set(data.day))])#, rotation=90)
        #lower = set(data.city)
        #ax.fill_between([data.day.min(), data.day.max()], [1,1]
        matches_here = match_data[match_data.city==city]
        for day, match in zip(matches_here.day, matches_here.match):
            if day >= data.day.min() and day <= data.day.max():
                if connr[(connr.day==day) & (connr.city==city)].nr.max() < 5e2:
                    ax.text(day, ylimup, match, rotation=90, ha='center', va='top', fontsize=10)
                else:
                    ax.text(day, 1.2*ylimdown, match, rotation=90, ha='center', va='bottom', fontsize=10)
        day_start = vac_data[vac_data.city==city].day_start.iloc[0]
        day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
        if day_end > day_start:
            ax.fill_between([day_start, day_end], [ylimdown]*2, [ylimup]*2, color='gray', alpha=.25)
        ax.set_ylim([ylimdown, ylimup])

        for holi in holis:
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        if city in ['Frankfurt am Main','München','Köln','Düsseldorf','Dortmund','Gelsenkirchen','Stuttgart']:
            holi = date(2024,5,30)
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        ax.set_yscale('log')

grid.add_legend(title='category of contact')#, bbox_to_anchor=(1, 1), loc="center left")
lg = grid._legend
for tx in lg.texts:
    if tx.get_text() == '1':
        tx.set_text('recurrent')
    elif tx.get_text() == '0':
        tx.set_text('random')

# Adjust the arrangement of the plots
#grid.fig.tight_layout(w_pad=1)

#plt.savefig(f'plots/fig4_{cdef_alt}/08_stable_random_contacts_cities_number.jpg', bbox_inches='tight', dpi=300)
#plt.savefig(f'plots/fig4_{cdef_alt}/08_stable_random_contacts_cities_number.pdf', bbox_inches='tight')
plt.show()

In [ ]:
sns.set_theme(style="ticks")

# Initialize a grid of plots with an Axes for each walk
grid = sns.FacetGrid(condur, col="city", hue="stability", hue_order=[1,0], palette=sns.husl_palette(2),#sns.color_palette(),#"tab20c",
                     col_wrap=1, height=2, aspect=5.5, row_order=city_list, legend_out=True)

# Draw a horizontal line to show the starting point
#grid.refline(y=0, linestyle=":")

# Draw a line plot to show the trajectory of each random walk
grid.map(plt.plot, "day", "dur", marker="o", ms=1)

# Adjust the tick positions and labels
#grid.set(xticks=[d for d in sorted(set(d2n.day)) if d.weekday()==5 and d.isocalendar()[1]%2==0],# yticks=[],
#         xlim=(d2n.day.min()-timedelta(3), d2n.day.max()+timedelta(3)), ylim=(max(.5,d2n.stime.min()), d2n.stime.max()))

ylimup, ylimdown = 1.2*condur.dur.max(), 1./1.2*condur.dur.min()
axes = grid.axes.flat
holis = [date(2024,5,1), date(2024,5,9), date(2024,5,20)]
for ax, city in zip(axes, city_list):
    if True:#for ax in ax_row:
        ax.set_ylabel('contact duration')
        #ax.plot([data_here.day.min(), data_here.day.max()], [1,1], c='gray')
        ax.set_title(city)
        ax.set_xticks(list(set(data.day)))
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in list(set(data.day))])#, rotation=90)
        #lower = set(data.city)
        #ax.fill_between([data.day.min(), data.day.max()], [1,1]
        matches_here = match_data[match_data.city==city]
        for day, match in zip(matches_here.day, matches_here.match):
            if day >= data.day.min() and day <= data.day.max():
                if condur[(condur.day==day) & (condur.city==city)].dur.max() < 5e2:
                    ax.text(day, ylimup, match, rotation=90, ha='center', va='top', fontsize=10)
                else:
                    ax.text(day, 1.2*ylimdown, match, rotation=90, ha='center', va='bottom', fontsize=10)
        day_start = vac_data[vac_data.city==city].day_start.iloc[0]
        day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
        if day_end > day_start:
            ax.fill_between([day_start, day_end], [ylimdown]*2, [ylimup]*2, color='gray', alpha=.25)
        ax.set_ylim([ylimdown, ylimup])

        for holi in holis:
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        if city in ['Frankfurt am Main','München','Köln','Düsseldorf','Dortmund','Gelsenkirchen','Stuttgart']:
            holi = date(2024,5,30)
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        ax.set_yscale('log')

grid.add_legend(title='category of contact')#, bbox_to_anchor=(1, 1), loc="center left")
lg = grid._legend
for tx in lg.texts:
    if tx.get_text() == '1':
        tx.set_text('recurrent')
    elif tx.get_text() == '0':
        tx.set_text('random')

# Adjust the arrangement of the plots
#grid.fig.tight_layout(w_pad=1)

#plt.savefig(f'plots/fig4_{cdef_alt}/08_stable_random_contacts_cities_duration.jpg', bbox_inches='tight', dpi=300)
#plt.savefig(f'plots/fig4_{cdef_alt}/08_stable_random_contacts_cities_duration.pdf', bbox_inches='tight')
plt.show()

In [ ]:
conavg = connr.merge(condur, on=['day','city','stability'])
conavg['avg'] = conavg.dur / conavg.nr
conavg

In [ ]:
sns.set_theme(style="ticks")

# Initialize a grid of plots with an Axes for each walk
grid = sns.FacetGrid(conavg, col="city", hue="stability", hue_order=[1,0], palette=sns.husl_palette(2),#sns.color_palette(),#"tab20c",
                     col_wrap=1, height=2, aspect=5.5, row_order=city_list, legend_out=True)

# Draw a horizontal line to show the starting point
#grid.refline(y=0, linestyle=":")

# Draw a line plot to show the trajectory of each random walk
grid.map(plt.plot, "day", "avg", marker="o", ms=1)

# Adjust the tick positions and labels
#grid.set(xticks=[d for d in sorted(set(d2n.day)) if d.weekday()==5 and d.isocalendar()[1]%2==0],# yticks=[],
#         xlim=(d2n.day.min()-timedelta(3), d2n.day.max()+timedelta(3)), ylim=(max(.5,d2n.stime.min()), d2n.stime.max()))

ylimup, ylimdown = 1.2*conavg.avg.max(), 1./1.2*conavg.avg.min()
axes = grid.axes.flat
holis = [date(2024,5,1), date(2024,5,9), date(2024,5,20)]
for ax, city in zip(axes, city_list):
    if True:#for ax in ax_row:
        ax.set_ylabel('rel. duration per contact')
        #ax.plot([data_here.day.min(), data_here.day.max()], [1,1], c='gray')
        ax.set_title(city)
        ax.set_xticks(list(set(data.day)))
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in list(set(data.day))])#, rotation=90)
        #lower = set(data.city)
        #ax.fill_between([data.day.min(), data.day.max()], [1,1]
        matches_here = match_data[match_data.city==city]
        for day, match in zip(matches_here.day, matches_here.match):
            if day >= data.day.min() and day <= data.day.max():
                if True:#conavg[(conavg.day==day) & (conavg.city==city)].avg.max() < 5e2:
                    ax.text(day, ylimup, match, rotation=90, ha='center', va='top', fontsize=10)
                else:
                    ax.text(day, 1.2*ylimdown, match, rotation=90, ha='center', va='bottom', fontsize=10)
        day_start = vac_data[vac_data.city==city].day_start.iloc[0]
        day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
        if day_end > day_start:
            ax.fill_between([day_start, day_end], [ylimdown]*2, [ylimup]*2, color='gray', alpha=.25)
        ax.set_ylim([ylimdown, ylimup])

        for holi in holis:
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        if city in ['Frankfurt am Main','München','Köln','Düsseldorf','Dortmund','Gelsenkirchen','Stuttgart']:
            holi = date(2024,5,30)
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        #ax.set_yscale('log')

grid.add_legend(title='category of contact')#, bbox_to_anchor=(1, 1), loc="center left")
lg = grid._legend
for tx in lg.texts:
    if tx.get_text() == '1':
        tx.set_text('recurrent')
    elif tx.get_text() == '0':
        tx.set_text('random')

# Adjust the arrangement of the plots
#grid.fig.tight_layout(w_pad=1)

#plt.savefig(f'plots/fig4_{cdef_alt}/08_stable_random_contacts_cities_durationpercontact.jpg', bbox_inches='tight', dpi=300)
#plt.savefig(f'plots/fig4_{cdef_alt}/08_stable_random_contacts_cities_durationpercontact.pdf', bbox_inches='tight')
plt.show()

## random vs. stable contacts and home distances

In [ ]:
dayh = [date(2024,7,17),date(2024,7,18),date(2024,7,19)]
dist_cmp_pos = data[(data.city=='Gelsenkirchen') & (data.day.isin(dayh))]
dist_cmp_pos['event'] = True
dist_cmp_neg = data[(data.city=='Gelsenkirchen') & (data.day.isin([d-timedelta(7) for d in dayh]))]
dist_cmp_neg['event'] = False
dist_cmp = pd.concat([dist_cmp_pos, dist_cmp_neg]).merge(randstab, on='pair')
daily_dur = dist_cmp.groupby(['day','pair']).tt2.apply(lambda x: len(set(x))).reset_index().rename(columns={'tt2':'dur'})
dist_cmp = dist_cmp.merge(daily_dur, on=['day','pair'])
dist_cmp

In [ ]:
dist_cmp = data.merge(match_data[['day','city','area_id']], on=['day','city'], how='left')
dist_cmp['event'] = ~dist_cmp.area_id.isna()
dist_cmp = dist_cmp.merge(randstab, on='pair')
daily_dur = dist_cmp.groupby(['day','pair']).tt2.apply(lambda x: len(set(x))).reset_index().rename(columns={'tt2':'dur'})
dist_cmp = dist_cmp.merge(daily_dur, on=['day','pair'])
dist_cmp

In [ ]:
match_base = match_data[['day','city','area_id']]
match_base['day'] = [d-timedelta(7) for d in match_base.day]
match_base['area_id'] = np.nan

In [ ]:
dist_cmp_pos = data.merge(match_data[['day','city','area_id']], on=['day','city'])
dist_cmp_pos['event'] = True
dist_cmp_neg = data.merge(match_base[['day','city','area_id']], on=['day','city'])
dist_cmp_neg['event'] = False
dist_cmp = pd.concat([dist_cmp_pos, dist_cmp_neg]).merge(randstab, on='pair')
daily_dur = dist_cmp.groupby(['day','pair']).tt2.apply(lambda x: len(set(x))).reset_index().rename(columns={'tt2':'dur'})
dist_cmp = dist_cmp.merge(daily_dur, on=['day','pair'])
dist_cmp

In [ ]:
dist_cmp['dhome1'], dist_cmp['dhome2'] = np.where(dist_cmp['dhome1'] > dist_cmp['dhome2'],\
                                                  (dist_cmp['dhome2'], dist_cmp['dhome1']),\
                                                  (dist_cmp['dhome1'], dist_cmp['dhome2']))
dist_cmp['dhome1'] /= 1e3
dist_cmp['dhome2'] /= 1e3
dist_cmp['dur'] *= 1e1
dist_cmp

In [ ]:
# Draw a scatter plot while assigning point colors and sizes to different
# variables in the dataset
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
#clarity_ranking = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]
g = sns.scatterplot(x="dhome1", y="dhome2",
                hue="event", size="dur",
                style="stability",
                palette="ch:r=-.2,d=.3_r",
                #hue_order=clarity_ranking,
                sizes=(1, 80), linewidth=0,
                data=dist_cmp[dist_cmp.city=='Gelsenkirchen'], ax=ax)
ax.set_xscale('symlog', linthresh=1e-1)
ax.set_yscale('symlog', linthresh=1e-1)
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))

In [ ]:
g = sns.relplot(
    data=dist_cmp, x="dhome1", y="dhome2",
    col="event", hue="stability", size="dur",# style="day",
    kind="scatter",
    palette=sns.husl_palette(2),#"ch:r=-.2,d=.3_r",
    #hue_order=clarity_ranking,
    sizes=(.1, 25), linewidth=0,
)
for ax in g.axes.flat:
    ax.set_xscale('symlog', linthresh=1e-1)
    ax.set_yscale('symlog', linthresh=1e-1)
    ax.set_xlim([0, 2e3])
    ax.set_ylim([0, 2e3])
    ax.set_xlabel('distance of contact to home of device 1 [km]')
    ax.set_ylabel('distance of contact to home of device 2 [km]')
    ax.set_aspect('equal')
#g.add_legend(title='category of contact')#, bbox_to_anchor=(1, 1), loc="center left")
lg = g._legend
for tx in lg.texts:
    if tx.get_text() == 'stability':
        tx.set_text('category')
    elif tx.get_text() == '1':
        tx.set_text('recurrent')
    elif tx.get_text() == '0':
        tx.set_text('random')
    elif tx.get_text() == 'dur':
        tx.set_text('duration [min]')
g.tight_layout()

plt.savefig(f'plots/fig1_{cdef_alt}/08_stable_random_homedist_germany.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/08_stable_random_homedist_germany.pdf', bbox_inches='tight')
plt.show()

In [ ]:
g = sns.relplot(
    data=dist_cmp[dist_cmp.city=='Gelsenkirchen'], x="dhome1", y="dhome2",
    col="event", hue="stability", size="dur",# style="day",
    kind="scatter",
    palette=sns.husl_palette(2),#"ch:r=-.2,d=.3_r",
    #hue_order=clarity_ranking,
    sizes=(.1, 25), linewidth=0,
)
for ax in g.axes.flat:
    ax.set_xscale('symlog', linthresh=1e-1)
    ax.set_yscale('symlog', linthresh=1e-1)
    ax.set_xlim([0, 2e3])
    ax.set_ylim([0, 2e3])
    ax.set_xlabel('distance of contact to home of device 1 [km]')
    ax.set_ylabel('distance of contact to home of device 2 [km]')
    ax.set_aspect('equal')
#g.add_legend(title='category of contact')#, bbox_to_anchor=(1, 1), loc="center left")
lg = g._legend
for tx in lg.texts:
    if tx.get_text() == 'stability':
        tx.set_text('category')
    elif tx.get_text() == '1':
        tx.set_text('recurrent')
    elif tx.get_text() == '0':
        tx.set_text('random')
    elif tx.get_text() == 'dur':
        tx.set_text('duration [min]')
g.tight_layout()

plt.savefig(f'plots/fig1_{cdef_alt}/08_stable_random_homedist_city.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/08_stable_random_homedist_city.pdf', bbox_inches='tight')
plt.show()